# Chapter13 연산자 오버로딩: 제대로 하기

### 13.1 단항 연산자

In [1]:
from array import array
import numbers
import reprlib
import math
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
        
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
#         hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
     
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return Vector(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg(cls=cls))
        return self._components[index]
    
    shortcut_name = 'xyzt'
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_name.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_name:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
                
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octests[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
vec = Vector([1, 2, 3])
abs(vec)

3.7416573867739413

In [2]:
-vec

Vector([-1.0, -2.0, -3.0])

In [3]:
+vec

Vector([1.0, 2.0, 3.0])

In [4]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [5]:
one_third == +one_third

True

In [6]:
ctx.prec = 28
one_third == +one_third

False

In [7]:
+one_third

Decimal('0.3333333333333333333333333333')

In [8]:
from collections import Counter

ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [9]:
ct['r'] = -3
ct['d'] = 0
ct

Counter({'a': 5, 'b': 2, 'r': -3, 'c': 1, 'd': 0})

In [10]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

### 벡터를 더하기 위해 + 오버로딩하기

In [11]:
def add(self, other):
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs)

Vector.__add__ = add

v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [12]:
v1 + v2 == Vector([3+6, 4+7, 8+5])

True

In [13]:
v1 = Vector([3, 4, 5, 6])
v2 = Vector([1, 2])
v1 + v2

Vector([4.0, 6.0, 5.0, 6.0])

In [14]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [15]:
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1

TypeError: can only concatenate tuple (not "Vector") to tuple

In [16]:
def radd(self, other):
    return self + other

Vector.__radd__ = radd

In [17]:
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [18]:
v1 + 1

TypeError: zip_longest argument #2 must support iteration

In [19]:
v1 + 'ABC'

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [20]:
def add(self, other):
    try:
        pairs = itertools.zip_longest(self, other)
        return Vector(a + b for a, b in pairs)
    except TypeError:
        return NotImplemented
    
Vector.__add__ = add

In [21]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [22]:
v1 + 'ABC'

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'

### 13.4 벡터를 스칼라와 곱하기 위해 * 오버로딩하기

In [23]:
def mul(self, scalar):
    return Vector(n * scalar for n in self)

def rmul(self, scalar):
    return self * scalar

Vector.__mul__ = mul
Vector.__rmul__ = rmul

In [24]:
v1 = Vector([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [26]:
11 * v1

Vector([11.0, 22.0, 33.0])

In [27]:
def mul(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented
    
Vector.__mul__ = mul

In [28]:
v1 = Vector([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [29]:
11 * v1

Vector([11.0, 22.0, 33.0])

In [30]:
v1 * [10.]

TypeError: can't multiply sequence by non-int of type 'Vector'

In [31]:
v1 * True 

Vector([1.0, 2.0, 3.0])

In [32]:
from fractions import Fraction
v2 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666])

In [33]:
Fraction.__mro__

(fractions.Fraction,
 numbers.Rational,
 numbers.Real,
 numbers.Complex,
 numbers.Number,
 object)

In [34]:
def matmul(self, other):
    try:
        return sum(a * b for a, b in zip(self, other))
    except TypeError:
        return NotImplemented
    
def rmatmul(self, other):
    return self @ other

Vector.__matmul__ = matmul
Vector.__rmatmul__ = rmatmul

In [35]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz == 38

True

In [36]:
[10, 20, 30] @ vz

380.0

## 13.5 향상된 비교 연산자

In [45]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [39]:
vc = Vector([1, 2])
va == vc

False

In [40]:
t3 = (1, 2, 3)
va == t3

True

In [41]:
[1, 2] == (1, 2)

False

In [43]:
def eq(self, other):
    if isinstance(other, Vector):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    else:
        return NotImplemented

Vector.__eq__ = eq

In [46]:
va == t3

False

In [47]:
va != vb

False

In [48]:
va != t3

True

## 13.6 복합 할당 연산자

In [56]:
v1 = Vector([1, 2, 3])
v1_alias = v1
id(v1)

4465437776

In [57]:
v1 += Vector([4, 5, 6])
v1

Vector([5.0, 7.0, 9.0])

In [58]:
id(v1)

4465437552

In [59]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [60]:
v1 *= 11
v1

Vector([55.0, 77.0, 99.0])

In [61]:
id(v1)

4465438056

In [65]:
import abc
import random
    
class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """ add iterable instance"""
    
    @abc.abstractmethod
    def pick(self):
        """ random selection
        raise LookupError if this is empty
        """ 
        
    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
            
        self.load(items)
        return tuple(sorted(items))

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()
        
class AddableBingoCage(BingoCage):
    
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
        
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = 'right operand in += must be {!r} or an iterable'
                raise TypeError(msg.format(self_cls))
                
        self.load(other_iterable)
        return self

In [67]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [68]:
globe.pick() in vowels

True

In [69]:
len(globe.inspect())

4

In [71]:
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
len(globe3.inspect())

7

In [72]:
void = globe + [10, 20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [73]:
globe_orig = globe
len(globe.inspect())

4

In [74]:
globe += globe2
len(globe.inspect())

7

In [75]:
globe += ['M', 'N']
len(globe.inspect())

9

In [76]:
globe is globe_orig

True

In [77]:
globe += 1

TypeError: right operand in += must be 'AddableBingoCage' or an iterable